In [1]:
import pandas as pd
import os

In [49]:
pd.set_option('display.max_columns', None)

In [106]:
dataftame = "senora_II.tsv"
folder = "./"

In [113]:
folder_tab = os.path.abspath(folder)
folder_tab

'/home/laura/DEVELOP/VCF_DDTB/old'

In [108]:
final_ddbb = pd.read_csv(dataftame, sep="\t",header=0)

In [109]:
def retrieve_tabs(sample_list, folder_tab):
    dict_tab_files = {}
    for sample in sample_list:
        for root, _, files in os.walk(folder_tab):
            for name in files:
                filename = os.path.join(root, name)
                if (sample in filename) and filename.endswith(".tab"):
                    dict_tab_files[sample] = pd.read_csv(filename, sep="\t",header=0)
    if len(sample_list) == len(dict_tab_files.keys()):
        return dict_tab_files
    else:
        print('Some tab files are missing, please make sure all samples are represented')
        sys.exit(1)

In [110]:
def variant_is_present(vcf_df,position):
    if position in vcf_df.POS.tolist():
        index_position = vcf_df.index[vcf_df.POS == position][0]
        if vcf_df.loc[index_position,'ALT_AD'] > vcf_df.loc[index_position,'REF_AD']:
            return "1"
        else:
            return "0"
    else:
        return "0"

In [ ]:
def recallibrate_ddbb(snp_matrix_ddbb, folder_tab):
    
    folder_tab = os.path.abspath(folder_tab)
    df = snp_matrix_ddbb
    #df = pd.read_csv(snp_matrix_ddbb, sep="\t",header=0)
    sample_list = df.columns[3:]
    n_samples = len(sample_list)
    
    all_samples_tab = retrieve_tabs(sample_list,folder_tab)
    
    for sample in sample_list:
        for index, _ in df[df.N < n_samples].iterrows():
            previous_snp = df.loc[index,sample]
            #df.loc[index,sample] coordinates to replace
            #all_samples_tab[sample] Dictionary with tab dataframes with all samples and variants
            #df.iloc[index,0] First columnn = POSITION to check with function and return new 0 or 1
            df.loc[index,sample] = variant_is_present(all_samples_tab[sample], df.iloc[index,0])
            #Substitute previous count (N) and list of samples
            if previous_snp == 0 and df.loc[index,sample] == 1:
                #Reassign number of samples (colimn with index 1)
                df.iloc[index,1] = df.iloc[index,1] + 1
                #Reassign list of samples (colimn with index 2)
                df.iloc[index,2] = df.iloc[index,2] + "," + sample
    return df

In [111]:
df = pd.read_csv(dataftame, sep="\t",header=0)

In [ ]:
df_recal = recallibrate_ddbb(df, folder_tab)

In [112]:
sample_list = df.columns[3:]
n_samples = len(sample_list)

In [114]:
all_samples_tab = retrieve_tabs(sample_list,folder_tab)

In [116]:
variant_is_present(all_samples_tab['AL10082989COL3'], 105007)

'0'

In [131]:
for sample in sample_list:
    for index, _ in df[df.N < n_samples].iterrows():
        previous_snp = df.loc[index,sample]
        position = df.iloc[index,0]
        #df.loc[index,sample] coordinates to replace
        #all_samples_tab[sample] Dictionary with tab dataframes with all samples and variants
        #df.iloc[index,0] First columnn = POSITION to check with function and return new 0 or 1
        post_snp = int(variant_is_present(all_samples_tab[sample], df.iloc[index,0]))
        df.loc[index,sample] = post_snp
        print(sample, position, previous_snp, post_snp)
        #df.loc[index,sample] = variant_is_present(all_samples_tab[sample], df.iloc[index,0])
        #post_snp = int(post_snp)
        #Substitute previous count (N) and list of samples
        if previous_snp == 0 and post_snp == 1:
            #Reassign number of samples (colimn with index 1)
            #print (str(df.iloc[index,1] + 1))
            df.iloc[index,1] = df.iloc[index,1] + 1
            #Reassign list of samples (colimn with index 2)
            #print(df.iloc[index,2] + "," + sample)
            df.iloc[index,2] = df.iloc[index,2] + "," + sample
        
    

AL10082989COL3 21819.0 0 0
AL10082989COL3 104915.0 1 1
AL10082989COL3 257448.0 1 1
AL10082989COL3 584511.0 0 1
AL10082989COL3 684671.0 0 0
AL10082989COL3 929094.0 0 0
AL10082989COL3 1167492.0 0 0
AL10082989COL3 1190017.0 0 1
AL10082989COL3 1191135.0 1 1
AL10082989COL3 1212076.0 1 1
AL10082989COL3 1341044.0 1 1
AL10082989COL3 1572851.0 0 1
AL10082989COL3 1636733.0 1 1
AL10082989COL3 1636742.0 1 1
AL10082989COL3 1636918.0 0 1
AL10082989COL3 1990919.0 1 1
AL10082989COL3 2306306.0 1 1
AL10082989COL3 2339719.0 1 1
AL10082989COL3 2339726.0 1 1
AL10082989COL3 2419142.0 1 1
AL10082989COL3 2637459.0 1 1
AL10082989COL3 2637502.0 1 1
AL10082989COL3 2637507.0 1 1
AL10082989COL3 2637520.0 1 1
AL10082989COL3 2637523.0 1 1
AL10082989COL3 2637541.0 1 1
AL10082989COL3 2719800.0 0 0
AL10082989COL3 2922984.0 1 1
AL10082989COL3 2941477.0 1 1
AL10082989COL3 2945167.0 1 1
AL10082989COL3 3379731.0 1 1
AL10082989COL3 3730896.0 1 1
AL10082989COL3 3750417.0 1 1
AL10082989COL3 3750421.0 1 1
AL10082989COL3 375080

In [100]:
#df['AL10082989COL3'][df.N < n_samples] = df_vcf['AL10082989COL3'].apply(lambda x: annotate_bed(all_samples_tab['AL10082989COL3'],x))

In [ ]:
#df['index_x10'] = df.apply(lambda row: 10*row.name, axis=1)

In [134]:
df[df.N > 2]

,Position,N,Samples,AL10082989COL3,AL10105494COL0
